# Importe

In [2]:
# general
import pandas as pd

# waits
import random
from time import sleep

# driver
import undetected_chromedriver.v2 as uc
from selenium import webdriver
import chromedriver_autoinstaller

# selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
# from ssl import Options

# multi
from concurrent import futures


# Driver

In [21]:
def get_driver(flag_proxy=False, free_proxies=""):
    # PATH = "C:\Program Files (x86)\chromedriver.exe"
    chrome_options = uc.ChromeOptions()

    # Removes navigator.webdriver flag
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    # prevent bugs and skip popups
    chrome_options.add_argument('--no-first-run --no-service-autorun --password-store=basic')
    chrome_options.add_argument("--disable-extensions")

    # uc - user agent
    # bot detection can determine if os in user-agent differs from real os
    # chrome_options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15")
    # chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

    # add multiple download flag
    dl_multi = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", dl_multi)

    # add download location
    dl_path = {"download.default_directory" : "C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\02_Download"}
    chrome_options.add_experimental_option("prefs", dl_path)

    # add profile
    uc.Chrome(user_data_dir='C:\\Users\\lange\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
    
    print("get_driver.counter: ", get_driver.counter)
    # add proxy if True
    # -1 to use first element after use of driver to scrape proxies
    if flag_proxy == True:
        i = get_driver.counter - 1
        PROXY = "109.202.219.167:8080"
        print("i (ip index): {0}\n ip: {1}\n PROXY: {2}".format(i, free_proxies[i], PROXY))
        # PROXY = str(free_proxies[i]) + ":" + 
        chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

    # be careful with headless browsing
    # chrome_options.add_argument("--headless")

    # webdriver
    # driver = webdriver.Chrome(executable_path=PATH, options=chrome_options)

    # undetected chromedriver
    driver = uc.Chrome(options=chrome_options)
    driver.maximize_window()
    
    # direkt zur erweiterten Suche spart Zeit
    # driver.get("https://www.handelsregister.de")
    # time.sleep(2)
    # link = driver.find_element_by_link_text("Erweiterte Suche")
    # link.click()
    get_driver.counter += 1
    return driver

get_driver.counter = 0

# Proxies

In [4]:
# chromedriver_autoinstaller.install() # To update your chromedriver automatically
# driver = webdriver.Chrome("C:\\Program Files (x86)\\chromedriver")

# Get free proxies for rotating
def get_free_proxies(driver):
    driver.get('https://sslproxies.org')

    table = driver.find_element(By.TAG_NAME, 'table')
    thead = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')
    tbody = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

    headers = []
    for th in thead:
        headers.append(th.text.strip())

    proxies = []
    for tr in tbody:
        proxy_data = {}
        tds = tr.find_elements(By.TAG_NAME, 'td')
        for i in range(len(headers)):
            proxy_data[headers[i]] = tds[i].text.strip()
        proxies.append(proxy_data)
    
    driver.quit()
    return proxies

# Scraper

In [44]:
for d in free_proxies:
   
    # now i is a dict, now we see the keys
    # of the dict
    for key, val in d.items():
       
        # print every val of each dict
        print(key, val)
 
    print("-------------")


IP Address 200.105.215.18
Port 33630
Code BO
Country Bolivia
Anonymity elite proxy
Google no
Https yes
Last Checked 19 secs ago
-------------
IP Address 151.181.91.10
Port 80
Code US
Country United States
Anonymity elite proxy
Google yes
Https yes
Last Checked 19 secs ago
-------------
IP Address 143.198.40.24
Port 8888
Code CA
Country Canada
Anonymity elite proxy
Google no
Https yes
Last Checked 19 secs ago
-------------
IP Address 198.59.191.234
Port 8080
Code US
Country United States
Anonymity elite proxy
Google no
Https yes
Last Checked 19 secs ago
-------------
IP Address 193.122.71.184
Port 3128
Code SA
Country Saudi Arabia
Anonymity anonymous
Google no
Https yes
Last Checked 10 mins ago
-------------
IP Address 172.105.184.208
Port 8001
Code AU
Country Australia
Anonymity elite proxy
Google no
Https yes
Last Checked 10 mins ago
-------------
IP Address 165.3.122.6
Port 80
Code US
Country United States
Anonymity anonymous
Google no
Https yes
Last Checked 10 mins ago
-------------

In [17]:
def scrape(swort, nsitz, free_proxies, flag_proxy=False):
    print(swort)
    print(nsitz)
    
    try:
        driver = get_driver(flag_proxy=flag_proxy, free_proxies=free_proxies)
        
        URL = "https://www.handelsregister.de/rp_web/erweitertesuche.xhtml"
        # URL = "https://bot.sannysoft.com/"    # test site for bot detection
        # URL = "https://nowsecure.nl"          # test site with max anti-bot protection
        driver.get(URL)
    except:
            
    # input
    input_swort = driver.find_element(By.ID, "form:schlagwoerter")
    input_nsitz = driver.find_element(By.ID, "form:NiederlassungSitz")
    input_swort.clear()
    input_nsitz.clear()

    input_swort.send_keys(swort)
    sleep(round(random.uniform(0.31, 2.11), 2))
    input_nsitz.send_keys(nsitz)
    sleep(round(random.uniform(0.31, 2.11), 2))

    # submit does not work, so click button
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(round(random.uniform(0.31, 2.11), 2))
    # btn_suche = driver.find_element_by_xpath(xpath="//*[contains(text(),'Suchen')]")
    driver.find_element(By.ID, "form:btnSuche").click()
    sleep(round(random.uniform(0.31, 2.11), 2))
    # extract and click clickable elements
    elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "AD")))
    sleep(round(random.uniform(0.31, 2.11), 2))
    
    index_elements = len(elements) - 1
    q = 0
    while q <= index_elements:
        try:
            elements[q].click()
            sleep(round(random.uniform(0.31, 2.11), 2))
            # download and go back
            driver.find_element(By.ID, "form:kostenpflichtigabrufen").click()
            driver.back()
            print("sucessfully downloaded {0} at index {1}".format(elements[q], q))
        except Exception as exc:
           print('swort {0} and nsitz {1} with q = {2} generated an exception: {3}'.format(swort, nsitz, q, exc))
        q += 1    
    else:
        driver.quit()

# Calls

In [37]:
# call proxy scraper
driver = get_driver()
free_proxies = get_free_proxies(driver)
free_proxies

[{'IP Address': '200.105.215.18',
  'Port': '33630',
  'Code': 'BO',
  'Country': 'Bolivia',
  'Anonymity': 'elite proxy',
  'Google': 'no',
  'Https': 'yes',
  'Last Checked': '19 secs ago'},
 {'IP Address': '151.181.91.10',
  'Port': '80',
  'Code': 'US',
  'Country': 'United States',
  'Anonymity': 'elite proxy',
  'Google': 'yes',
  'Https': 'yes',
  'Last Checked': '19 secs ago'},
 {'IP Address': '143.198.40.24',
  'Port': '8888',
  'Code': 'CA',
  'Country': 'Canada',
  'Anonymity': 'elite proxy',
  'Google': 'no',
  'Https': 'yes',
  'Last Checked': '19 secs ago'},
 {'IP Address': '198.59.191.234',
  'Port': '8080',
  'Code': 'US',
  'Country': 'United States',
  'Anonymity': 'elite proxy',
  'Google': 'no',
  'Https': 'yes',
  'Last Checked': '19 secs ago'},
 {'IP Address': '193.122.71.184',
  'Port': '3128',
  'Code': 'SA',
  'Country': 'Saudi Arabia',
  'Anonymity': 'anonymous',
  'Google': 'no',
  'Https': 'yes',
  'Last Checked': '10 mins ago'},
 {'IP Address': '172.105.184

In [7]:
# example df
data = [['INTENSE AG', 'Würzburg'], ['snapADDY GmbH', 'Würzburg']]
df_terms = pd.DataFrame(data, columns=['swort', 'nsitz'])

df_terms


,swort,nsitz
0,INTENSE AG,Würzburg
1,snapADDY GmbH,Würzburg


In [22]:
# call scraper
# free_proxies = [d['IP Address'] for d in free_proxies]
get_driver.counter = 1


# check different methods for faster applying functions to data
# https://www.ml4devs.com/articles/pandas-dataframe-apply-function-iterate-over-rows/
for index, row in df_terms.iterrows():
    scrape(swort=row['swort'], nsitz=row['nsitz'], flag_proxy=True, free_proxies=free_proxies)

INTENSE AG
Würzburg
get_driver.counter:  1
i (ip index): 0
 ip: 151.181.91.10
 PROXY: 109.202.219.167:8080


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="form:schlagwoerter"]"}
  (Session info: chrome=108.0.5359.22)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E1ACD3+2075859]
	Ordinal0 [0x00DAEE61+1633889]
	Ordinal0 [0x00CAB7BD+571325]
	Ordinal0 [0x00CDAC2F+764975]
	Ordinal0 [0x00CDAE1B+765467]
	Ordinal0 [0x00D0D0F2+970994]
	Ordinal0 [0x00CF7364+881508]
	Ordinal0 [0x00D0B56A+963946]
	Ordinal0 [0x00CF7136+880950]
	Ordinal0 [0x00CCFEFD+720637]
	Ordinal0 [0x00CD0F3F+724799]
	GetHandleVerifier [0x010CEED2+2769538]
	GetHandleVerifier [0x010C0D95+2711877]
	GetHandleVerifier [0x00EAA03A+521194]
	GetHandleVerifier [0x00EA8DA0+516432]
	Ordinal0 [0x00DB682C+1665068]
	Ordinal0 [0x00DBB128+1683752]
	Ordinal0 [0x00DBB215+1683989]
	Ordinal0 [0x00DC6484+1729668]
	BaseThreadInitThunk [0x76096739+25]
	RtlGetFullPathName_UEx [0x77048FD2+1218]
	RtlGetFullPathName_UEx [0x77048F9D+1165]


# Other

In [ ]:
# MULTI Threading
id, swort, nsitz = get_terms(terms=terms)
# default number of threads is optimized for cpu cores 
# possible to set with `max_workers` like `futures.ThreadPoolExecutor(max_workers=...)`
with futures.ThreadPoolExecutor() as executor:     
    # store the id for each thread as a dict, so we can know which thread fails
    future_results = { id : executor.submit(scrape(swort, nsitz), terms) for id in terms }
    for id, future in future_results.items(): 
        try:        
           future.result() # can use `timeout` to wait max seconds for each thread  
        except Exception as exc: # can give a exception in some thread
           print('id {0} generated an exception: {1}'.format(id, exc))

In [ ]:
# PROXY TEST
PROXY = "174.138.16.96:8888" # IP:PORT or HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

driver = uc.Chrome(options=chrome_options)
driver.get("http://whatismyipaddress.com")